In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='6um'

In [4]:
design.get_chip_z('Q_chip')

'6um'

In [5]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='450um',
                                            pad_height='130um',
                                            pad_gap='100um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+110}um",
                                              pad_width="200um",
                                              pad_height="140um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [13]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '904um', '100um')
            

# EPR Analysis

In [14]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [15]:
hfss.start()

INFO 05:17PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:17PM [load_ansys_project]: 	Opened Ansys App
INFO 05:17PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 05:17PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   V8
INFO 05:17PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 05:17PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:17PM [connect]: 	Connected to project "V8" and design "Design_q3d" 😀 



True

In [16]:
hfss.activate_ansys_design("flip6um_V9", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

05:17PM 34s WARNING [activate_ansys_design]: The design_name=flip6um_V9 was not in active project.  Designs in active project are: 
['flip7um_V8', 'Design_q3d', 'flip7um_V8-', 'flip7um_V9'].  A new design will be added to the project.  
INFO 05:17PM [connect_design]: 	Opened active design
	Design:    flip6um_V9 [Solution type: Eigenmode]
WARNING 05:17PM [connect_setup]: 	No design setup detected.
WARNING 05:17PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:17PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [17]:
hfss.close()


True

# LOM Analysis

In [18]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [19]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='4.97',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '4.97',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [21]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:30PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpdxj8xcg4.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 1, False
INFO 05:30PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpwuwo4vw7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 1, False
INFO 05:30PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpd8lrbbad.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 2, False
INFO 05:30PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpsi940cgo.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 3, False
INFO 05:30PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpw44dfbw7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4970000000, Maxwell, 4, False
INFO 05:30

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,189.610,-16.095,0.000,-174.85
pad_top_Q0,-16.095,189.010,-41.972,-131.16
quad_coupler_c0,0.000,-41.972,157.930,-115.39
SignalNet1,-174.850,-131.160,-115.390,491.42


In [27]:
c1.setup.junctions = Dict({'Lj': 7, 'Cj': 2})
c1.setup.freq_readout = 6.4
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.586360 [GHz]
EC 178.253485 [MHz]
EJ 23.342229 [GHz]
alpha -192.740446 [MHz]
dispersion 0.000106 [KHz]
Lq 6.997177 [nH]
Cq 108.666757 [fF]
T1 33.889259 [us]

**Coupling Properties**

tCqbus1 -7.071149 [fF]
gbus1_in_MHz -74.773852 [MHz]
χ_bus1 -2.647380 [MHz]
1/T1bus1 4696.324148 [Hz]
T1bus1 33.889259 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,7.120089,295.403637,23.342229,-328.003132,0.124858,[-39.75002290665509],[-3.6770743308831073],3.677074,39.750023
2,5.584012,178.09839,23.342229,-192.56533,0.000105,[-39.394819940083984],[-0.7305093833134009],0.730509,39.394820
3,5.515215,173.585854,23.342229,-187.475048,0.00007,[-58.727423818249974],[-1.3723228969381063],1.372323,58.727424
4,5.524217,174.172764,23.342229,-188.136577,0.000074,[-68.90318876961874],[-1.9300054886930287],1.930005,68.903189
5,5.533577,174.78419,23.342229,-188.825906,0.000078,[-63.018811635307685],[-1.6510922442406142],1.651092,63.018812
6,5.543657,175.443896,23.342229,-189.569858,0.000083,[-63.54760040218892],[-1.720349139964493],1.720349,63.547600
7,5.5497,175.840046,23.342229,-190.016691,0.000086,[-69.18276455272888],[-2.0692143935340814],2.069214,69.182765
8,5.544498,175.498968,23.342229,-189.631971,0.000083,[-72.183871868181],[-2.2242603311170295],2.224260,72.183872
9,5.549654,175.837019,23.342229,-190.013276,0.000085,[-72.39240107513935],[-2.265409371475117],2.265409,72.392401
10,5.58636,178.253485,23.342229,-192.740446,0.000106,[-74.7738515493293],[-2.6473795751119154],2.647380,74.773852


In [28]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
